# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0702 15:03:05.795240  1384 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0702 15:03:06.059529  1384 deprecation_wrapper.py:119] From F:\Instalaciones\Anaconda3\envs\privateai\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{14045917382: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

14045917382

In [15]:
x.id

60596474804

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:60596474804 -> bob:14045917382]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# try this project here!

In [22]:
alice = sy.VirtualWorker(hook, id = "alice")

In [23]:
def send_tensor_to_workers(tensor, workers =  []):
    y = tensor.send(*workers)
    return  y

def get_tensor(tensor):
    y = tensor.get()
    return y
    

In [24]:
y = th.Tensor([10,2,3,4])
workers = [bob,alice]
y = send_tensor_to_workers(y, workers =workers )

for worker in workers:
    print("Objects of worker {} :{} ".format(worker.id, worker._objects))

y = get_tensor(y)

print("")
print("Tensor:{}".format(y))

Objects of worker bob :{56862499222: tensor([10.,  2.,  3.,  4.])} 
Objects of worker alice :{98261450239: tensor([10.,  2.,  3.,  4.])} 

Tensor:[tensor([10.,  2.,  3.,  4.]), tensor([10.,  2.,  3.,  4.])]


# Lesson: Introducing Remote Arithmetic

In [162]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [163]:
x

(Wrapper)>[PointerTensor | me:66153678830 -> bob:66432643589]

In [164]:
y

(Wrapper)>[PointerTensor | me:70822625530 -> bob:1964130973]

In [165]:
z = x + y

In [166]:
z

(Wrapper)>[PointerTensor | me:15252398824 -> bob:92562270702]

In [167]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [168]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:41217911574 -> bob:8712949788]

In [169]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [174]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [176]:
z = (x + y).sum()


tensor(20., requires_grad=True)

In [172]:
z.backward()

(Wrapper)>[PointerTensor | me:32755830893 -> bob:57301571289]

In [173]:
x = x.get()

In [37]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [38]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [311]:
a,b = th.Tensor([[1,1,1]]),th.Tensor([[2,2,2]])
print(a.shape,b.shape)
a+b


a = th.randn(10)
b = th.randn(10)
c = a * b

d = th.tensor([2.0])
print(a)
print(c.shape)
print(d)

print(a*d)

torch.Size([1, 3]) torch.Size([1, 3])
tensor([-1.0116,  1.3143,  0.1225,  0.7255,  0.8620, -0.8590, -0.3879,  0.6767,
         1.6156, -1.3650])
torch.Size([10])
tensor([2.])
tensor([-2.0232,  2.6286,  0.2449,  1.4511,  1.7239, -1.7180, -0.7758,  1.3534,
         3.2312, -2.7300])


# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [352]:
# try this project here!

In [353]:
import torch as th
from torch.autograd import Variable
X = Variable(th.Tensor(
                 [1,2,3]
             ))

X = X.view(X.shape[0],1)

y = Variable(th.Tensor(
                 [2,4,6]
             ))
y = y.view(y.shape[0],1)


(X*y).shape

torch.Size([3, 1])

### Send Tensors to Bob

In [354]:
X = X.send(bob)
y = y.send(bob)

## Model Definition

In [355]:
class LinearRegression():
    
    def __init__(self, shape):
        
        #Can initialize weigths with zeros, not on neural nets.
        self.w = th.zeros((1,), requires_grad = True) #th.randn(*x.shape)
        #send to bob
        self.w =  self.w.send(bob)
        
        self.b = th.zeros(1, requires_grad = True)
        #send to bob
        self.b = self.b.send(bob)
        
    
    
    def forward(self,x):
        print("Shapes", X.shape,self.w.shape)
        y = (X * self.w) + self.b
        
        return y
    
    def backward(self, loss):
        """
        y: torch Tensor
        recieves outouput of linear model
        return gradient with respect to weights
        """
        loss.backward()
        
        
    def gradient_step(self,learning_rate):
        
        assert self.w.grad  is not None and  self.b.grad is not None,"Compute gradients first."
        
        self.w = self.w - learning_rate * self.w.grad
        self.b = self.b - learning_rate * self.b.grad
        
    def l2_norm(self,x,y):
        """
        x: torch tensor
        y:torch tensor
        compute and return l-2 norm between x and y:
        """
        
        diff = x - y
        print("Shape of diff", diff.shape)
        
        
        
        return (1/2)*th.matmul(diff.t(), diff)

In [356]:
model = LinearRegression(X.shape)

In [357]:
#Make prediction
y_pred = model.forward(X)
print("Y prediction", y_pred)

#Compute cost function
loss = model.l2_norm(X, y_pred)

#Compute gradients of loss with respect to parameters w,b
model.backward(loss)


print("Gradients:")
print("for w:")
w_grad = model.w.grad.clone().get()
print(w_grad)

print("for b:")
b_grad = model.b.grad.clone().get()
print(b_grad)

#get params causes to left bob
#print("Parameters w: {} , b: {}".format(model.w.get(),model.b.get()))

#Apply gradient descent step
model.gradient_step(learning_rate = 0.05)



"""
for i in range(10):
    y_pred = model.forward(X)
    print("Y prediction", y_pred)

    #Compute cost function
    loss = model.l2_norm(X, y_pred)
    print("Loss:", loss)
    #send to bob
    #loss.send(bob)


    #Compute gradients of loss with respect to parameters w,b
    model.backward(loss)


    print("Gradients:")
    print("for w:")

    w_grad = model.w.grad.clone().get()
    print(w_grad)

    print("for b:")
    b_grad = model.b.grad.clone().get()
    print(b_grad)

#get params causes to left bob    
print("Parameters w: {} , b: {}".format(model.w.get(),model.b.get()))
"""

Shapes torch.Size([3, 1]) torch.Size([1])
Y prediction (Wrapper)>[PointerTensor | me:1104372877 -> bob:47567548741]
Shape of diff torch.Size([3, 1])
Gradients:
for w:
tensor([-14.])
for b:
tensor([-6.])


'\nfor i in range(10):\n    y_pred = model.forward(X)\n    print("Y prediction", y_pred)\n\n    #Compute cost function\n    loss = model.l2_norm(X, y_pred)\n    print("Loss:", loss)\n    #send to bob\n    #loss.send(bob)\n\n\n    #Compute gradients of loss with respect to parameters w,b\n    model.backward(loss)\n\n\n    print("Gradients:")\n    print("for w:")\n\n    w_grad = model.w.grad.clone().get()\n    print(w_grad)\n\n    print("for b:")\n    b_grad = model.b.grad.clone().get()\n    print(b_grad)\n\n#get params causes to left bob    \nprint("Parameters w: {} , b: {}".format(model.w.get(),model.b.get()))\n'

In [349]:
x_ =th.Tensor([1,2,3])
x_ = x_.view(1,x_.shape[0])
x_ = x_.send(bob)
y = model.forward( x_  )
y.get()

Shapes torch.Size([3, 1]) torch.Size([1])


tensor([[1.0000],
        [1.7000],
        [2.4000]], requires_grad=True)

# Lesson: Garbage Collection and Common Errors


In [44]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [86]:
x = "asdf"

In [87]:
bob._objects

{17874592902: tensor([1, 2, 3, 4, 5]),
 61081092152: tensor([1, 1, 1, 1, 1]),
 84265830872: tensor(20., grad_fn=<SumBackward0>)}

In [88]:
del x

In [89]:
bob._objects

{17874592902: tensor([1, 2, 3, 4, 5]),
 61081092152: tensor([1, 1, 1, 1, 1]),
 84265830872: tensor(20., grad_fn=<SumBackward0>)}

In [90]:
bob = bob.clear_objects()
bob._objects

{}

In [91]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [92]:
bob._objects

{89861956797: tensor([1, 2, 3, 4, 5])}

In [93]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [94]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:40457233960 -> bob:2484718012]
Tensor B: tensor([1, 1, 1, 1, 1])

In [130]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [69]:
from torch import nn, optim

In [67]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [70]:
# A Toy Model
model = nn.Linear(2,1)

In [71]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [89]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [90]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [91]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [122]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [123]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [163]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [167]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
x = x.send(alice)

In [171]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [172]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [174]:
y = x + x

In [175]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [176]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [177]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [178]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [190]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [212]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [213]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [214]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [215]:
alice._objects

{}

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}